In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00


In [3]:
import sklearn
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import DistilBertTokenizerFast, TFAutoModel
from sklearn.model_selection import train_test_split

In [4]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
headphones_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AspectWise/Headphones_custom_dataset.csv')

In [7]:
shuffled_df = headphones_df.sample(frac=1)
shuffled_df.dropna(inplace=True)
shuffled_df

,Unnamed: 0,Reviews,Sentiment
1140,1140,I was disappointed with the sound quality o...,10
1635,408,These JBL headphones are incredible The...,15
230,230,The Bose headphones have a lot of creaking ...,2
1713,486,Save your money and avoid these overprice...,16
830,830,The seams on this Sony headphone are so p...,8
...,...,...,...
1879,652,These JBL headphones are a great deal for...,17
16,16,The Bose headphones have a lot of sound lea...,0
506,506,I was pleasantly surprised by the afforda...,5
1027,1027,Exceptional sound quality for the price ...,11


In [8]:
shuffled_df['Text_length'] = shuffled_df['Reviews'].apply(len)
shuffled_df = shuffled_df[shuffled_df['Text_length'] > 1]

In [9]:
textlen = shuffled_df['Text_length'].tolist()
1 in textlen

False

In [10]:
X = shuffled_df['Reviews'].tolist()
y = shuffled_df['Sentiment'].tolist()
# y = pd.get_dummies(shuffled_bose_df['Sentiment']).values
len(X), len(y)

(1854, 1854)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
X_eval, X_test, y_eval, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [13]:
while " " in X_train:
    X_train.remove(" ")
while " " in X_test:
    X_test.remove(" ")

In [14]:
X_train_encodings = tokenizer(X_train, truncation=True, padding=True)
X_test_encodings = tokenizer(X_test, truncation=True, padding=True)
X_eval_encodings = tokenizer(X_eval, truncation=True, padding=True)

In [15]:
eval_dataset = tf.data.Dataset.from_tensor_slices((dict(X_eval_encodings), y_eval))

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_encodings), y_train))

In [17]:
training_args = TFTrainingArguments(
    output_dir = '/results',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir = '/logs',
    logging_steps=10,
    eval_steps=100,
)

In [18]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=18)

    trainer = TFTrainer(
      model=model,
      args = training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
    )

    trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_transform', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [19]:
result = trainer.evaluate()
result

{'eval_loss': 0.14515956554537504}

In [20]:
save_directory = '/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model_full'
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model_full/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model_full/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model_full/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model_full/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model_full/tokenizer.json')

In [21]:
save_directory = '/content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model'
tokenizer_fine_tuned = DistilBertTokenizerFast.from_pretrained(save_directory)
model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/AspectWise/combined_model and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [22]:
test_encodings = tokenizer(
    X_test,
    truncation=True,
    padding=True,
    return_tensors='tf',
)

In [23]:
testing_data = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))

In [24]:
final_pred = trainer.predict(testing_data)

In [25]:
final_pred

PredictionOutput(predictions=array([[-1.348332  , -3.0140092 , -2.6291466 , ..., -2.750393  ,
        -1.3157481 , -2.3230648 ],
       [-2.417282  , -2.8568401 , -1.9917054 , ..., -1.268404  ,
        -1.157404  , -2.4465153 ],
       [-1.9517373 , -3.598149  , -2.710924  , ..., -5.37343   ,
        -1.8657684 , -3.6438613 ],
       ...,
       [ 1.5908066 , -0.12263421,  6.1709814 , ..., -5.210027  ,
        -5.1556706 , -5.6365795 ],
       [-4.841501  , -5.4849577 , -5.162939  , ..., -1.0647718 ,
         0.1603304 ,  7.38736   ],
       [-2.88683   , -0.7955918 , -2.2788312 , ..., -5.2217865 ,
        -4.1711802 , -2.7779284 ]], dtype=float32), label_ids=array([12, 14, 10,  5, 12,  9,  6,  8,  2, 12,  2, 16, 17, 13, 16,  8,  2,
        1,  6, 15,  7,  1,  9,  2,  1, 16,  5,  7,  3, 15, 15, 15,  7, 12,
        6,  6,  1,  2,  1,  6, 11, 13,  9, 15,  1,  5, 10,  7,  2,  8, 15,
        9, 17,  4,  5, 11, 11,  3, 17, 10, 10,  6, 14, 16,  4, 12, 13, 16,
       17,  5,  6,  6,  5,  0,  

In [26]:
final_prediction = tf.argmax(final_pred.predictions, axis=1)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(final_prediction, y_test)
print(cm)
cr = classification_report(final_prediction, y_test)
print(cr)
                           

[[ 5  0  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  8  1  0  0  0  0  0  0  0  0  0  0  2  0]
 [ 0  0  0  0  0 21  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 27  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 20  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 17  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 22  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 17  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 14  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 13  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 14  0]
 [ 0  0  0  0  0  0  0  0  0  0

In [28]:
def custom_input():
  resp = input("Enter your review : ")
  resp_token = tokenizer(resp, truncation=True, return_tensors = 'tf')
  custom_test = tf.data.Dataset.from_tensor_slices((dict(resp_token), [0]))
  preds = model_fine_tuned.predict(custom_test)
  print(preds)
  prediction = tf.argmax(preds.logits, axis=1)
  return prediction

In [30]:
prediction = custom_input().numpy()[0]
lst = ['Bose bad audio', 'Bose good audio', 'Bose bad build', 'Bose good build', 'Bose bad price', 'Bose good price', 
       'Sony bad audio', 'Sony good audio', 'Sony bad build',  'Sony good build', 'Sony bad price', 'Sony good price',
       'JBL bad audio', 'JBL good audio', 'JBL bad build',  'JBL good build', 'JBL bad price', 'JBL good price']
print(lst[prediction])

Enter your review : Sony has a bad build quality
1/1 [==============================] - 1s 1s/step
TFSequenceClassifierOutput(loss=None, logits=array([[-1.8015996 , -1.6142204 , -1.6931279 , -1.9748894 , -1.1043975 ,
        -1.896977  , -0.4660105 , -1.9643162 ,  3.8613298 ,  2.3088105 ,
        -0.43129352, -0.56484413]], dtype=float32), hidden_states=None, attentions=None)
Sony bad build
